# Softmax vs P-bit Sampler Comparison

Minimal demonstration comparing distributions (V=16).


In [ ]:
import sys
sys.path.insert(0, '..')

import torch
from qllm_pbits.token_sampler import PBitOneHotSampler, SoftmaxMultinomialSampler
from qllm_pbits.pbits.metrics import total_variation, kl_divergence

torch.manual_seed(42)
print("✓ Imports successful")


## Sample from both samplers


In [ ]:
V = 16
logits = torch.randn(V)
n_samples = 500

# Baseline
baseline_sampler = SoftmaxMultinomialSampler()
baseline_samples = torch.tensor([baseline_sampler.sample(logits) for _ in range(n_samples)])
baseline_hist = torch.bincount(baseline_samples, minlength=V).float()
baseline_dist = baseline_hist / baseline_hist.sum()

# P-bit
pbit_sampler = PBitOneHotSampler(lam=20.0, n_gibbs_steps=100)
pbit_samples = torch.tensor([pbit_sampler.sample(logits) for _ in range(n_samples)])
pbit_hist = torch.bincount(pbit_samples, minlength=V).float()
pbit_dist = pbit_hist / pbit_hist.sum()

# Metrics
tv = total_variation(baseline_dist, pbit_dist)
kl = kl_divergence(baseline_dist, pbit_dist)

print(f"TV distance: {tv:.4f}")
print(f"KL divergence: {kl:.4f} nats")
print(f"Invalid rate: {pbit_sampler.get_invalid_rate():.4f}")
print("\n✓ Comparison complete")
